In [2]:
import pandas as pd
import xarray as xr
import numpy as np
from oceanograpy.io import _variable_defs as vardef
from oceanograpy.util import time
import glob2
import matplotlib.pyplot as plt
import re
from typing import Optional
from tqdm.notebook import tqdm 
import cftime
from typing import Optional
from itertools import zip_longest
from matplotlib.dates import date2num

In [3]:
from oceanograpy.data.nc_format import _standard_attrs

In [6]:
import  matplotlib.pyplot as plt
from oceanograpy.data.nc_format import conventionalize#, _standard_attrs, check_conventions
from oceanograpy.data.ship_ctd_tools import _ctd_tools as tools#, _standard_attrs, check_conventions
from oceanograpy.data import ctd
from oceanograpy.io import sbe


In [7]:
btl_fn = './sta013_01.btl'

### Try loading a header

In [8]:
header = sbe.read_header(btl_fn)

In [14]:
header

{'col_nums': None,
 'col_names': ['Bottle Position',
  'Date Time',
  'Sal00',
  'Sbeox0ML/L',
  'Sigma-é00',
  'SvCM',
  'PrDM',
  'T090C',
  'C0S/m',
  'Sbeox0V',
  'FlECO-AFL'],
 'col_longnames': None,
 'SN_info': None,
 'moon_pool': None,
 'SBEproc_hist': ['# datcnv_date = Jan 13 2021 09:16:32, 7.26.7.129\n',
  '# datcnv_in = C:\\Users\\julius.lauber\\Documents\\WORK\\TrollTransect20_21\\Processing\\CTD\\Raw\\sta013_01.hex C:\\Users\\julius.lauber\\Documents\\WORK\\TrollTransect20_21\\Processing\\CTD\\Raw\\STA013_01.XMLCON\n',
  '# datcnv_ox_hysteresis_correction = yes\n',
  '# datcnv_bottle_scan_range_source = scans marked with bottle confirm bit, 0, 2\n',
  '# datcnv_scans_per_bottle = 49\n',
  '# bottlesum_date = Jan 13 2021 09:16:34, 7.26.7.129\n',
  '# bottlesum_in = C:\\Users\\julius.lauber\\Documents\\WORK\\TrollTransect20_21\\Processing\\CTD\\Processed\\sta013_01.ros C:\\Users\\julius.lauber\\Documents\\WORK\\TrollTransect20_21\\Processing\\CTD\\Raw\\STA019_01.XMLCON\n',
  

### Try loading the btl data and metadata (one file) to xr Dataset

In [9]:
ds = sbe.read_btl(btl_fn)

In [10]:
ds

<xarray.Dataset>
Dimensions:                 (NISKIN_NUMBER: 9, TIME: 1)
Coordinates:
  * NISKIN_NUMBER           (NISKIN_NUMBER) float64 1.0 2.0 3.0 ... 7.0 8.0 9.0
  * TIME                    (TIME) float64 1.863e+04
Data variables: (12/18)
    PSAL1                   (NISKIN_NUMBER) float64 34.67 34.68 ... 34.3 34.21
    DOXY1_instr             (NISKIN_NUMBER) float64 4.868 4.784 ... 7.294 7.509
    SIGTH1                  (NISKIN_NUMBER) float64 27.83 27.82 ... 27.61 27.54
    SVEL                    (NISKIN_NUMBER) float64 1.475e+03 ... 1.44e+03
    PRES                    (NISKIN_NUMBER) float64 1.518e+03 ... 25.27
    PRES_std                (NISKIN_NUMBER) float64 0.033 0.105 ... 0.083 0.088
    ...                      ...
    CHLA1_fluorescence      (NISKIN_NUMBER) float64 1.047 1.049 ... 1.66 2.962
    CHLA1_fluorescence_std  (NISKIN_NUMBER) float64 0.0 0.0081 ... 0.0081 0.0964
    TIME_SAMPLE             (NISKIN_NUMBER) float64 1.863e+04 ... 1.863e+04
    STATION                 (TIME) <U4 ' 012'
    LATITUDE                (TIME) float64 -69.42
    LONGITUDE               (TIME) float64 6.182
Attributes:
    history:              2021-01-04: Data collection.\n2021-01-13: Processed...
    ship:                 Malik Arctica
    cruise_name:          DML 2020
    station:               012
    latitude:             -69.4183
    longitude:            6.182
    start_time:           2021-01-04T14:51:19Z
    start_time_source:    "NMEA UTC" header line
    SBE_processing:       SBE SOFTWARE PROCESSING STEPS (extracted from .btl ...
    SBE_processing_date:  2021-01-13T09:16:32Z
    source_files:         STA013_01.HEX, STA013_01.XMLCON -> STA013_01.BTL

### Try loading and concatenating all .btl files from a cruise

In [12]:
path_to_btls = './'
D = ctd.dataset_from_btl_dir(path_to_btls)

['./sta013_01.btl', './sta015_01.btl', './sta014_01.btl']
Loaded 3 profiles from list of Datasets.


Joining profiles together:   0%|          | 0/3 [00:00<?, ?it/s]


NOTE: No cruise ID found in the dataset. Remember to assign!
-> ds = .set_ctd_attr(N, "cruise_name").


In [13]:
D

<xarray.Dataset>
Dimensions:                 (NISKIN_NUMBER: 11, TIME: 3)
Coordinates:
  * NISKIN_NUMBER           (NISKIN_NUMBER) float64 1.0 2.0 3.0 ... 10.0 11.0
  * TIME                    (TIME) float64 1.863e+04 1.863e+04 1.863e+04
Data variables: (12/19)
    PSAL1                   (NISKIN_NUMBER, TIME) float64 34.67 34.66 ... 33.96
    DOXY1_instr             (NISKIN_NUMBER, TIME) float64 4.868 4.998 ... 7.488
    SIGTH1                  (NISKIN_NUMBER, TIME) float64 27.83 27.84 ... 27.33
    SVEL                    (NISKIN_NUMBER, TIME) float64 1.475e+03 ... 1.44e+03
    PRES                    (NISKIN_NUMBER, TIME) float64 1.518e+03 ... 9.782
    PRES_std                (NISKIN_NUMBER, TIME) float64 0.033 0.032 ... 0.09
    ...                      ...
    CHLA1_fluorescence_std  (NISKIN_NUMBER, TIME) float64 0.0 0.0161 ... 0.0718
    TIME_SAMPLE             (NISKIN_NUMBER, TIME) float64 1.863e+04 ... 1.863...
    STATION                 (TIME) <U4 ' 012' ' 014' ' 015'
    LATITUDE                (TIME) float64 -69.42 -69.06 -69.25
    LONGITUDE               (TIME) float64 6.182 6.048 6.154
    CRUISE                  <U12 '!! CRUISE !!'
Attributes:
    history:         2021-01-04 to 2021-01-05: Data collection.\n2021-01-13: ...
    ship:            Malik Arctica
    cruise_name:     DML 2020
    latitude:        -69.4183
    longitude:       6.182
    SBE_processing:  SBE SOFTWARE PROCESSING STEPS (extracted from .btl file ...
    source_files:    E.g. STA013_01.HEX, STA013_01.XMLCON -> STA013_01.BTL
    featureType:     profile

### Align with CF-conventions 
(mostly adds a bunch of global attributes)

**Note** This should get us close, but for a typical dataset, we still have to add various custom attributes like title, summary, etc.

In [205]:
D = ctd.make_publishing_ready(D)

Dropped these variables from the Dataset: ['DOXY1_instr', 'SIGTH1', 'SVEL', 'PRES_std', 'TEMP1_std', 'CNDC1_std', 'OXYV1', 'OXYV1_std', 'CHLA1_instr', 'CHLA1_instr_std', 'TIME_SAMPLE'].


### Inspect the file

printing the dataset so we can browse the file including global and variable attributes

#### Run a convention checker 

(Checks CF-1.8 and ACDD-1.3)

In [206]:
ctd.check_conventions.check_file(D)



--------------------------------------------------------------------------------
                         IOOS Compliance Checker Report                         
                                 Version 5.1.0                                  
                     Report generated 2023-11-20T21:39:35Z                      
                                     cf:1.8                                     
http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.html
--------------------------------------------------------------------------------
                               Corrective Actions                               
temp.nc has 1 potential issue


                                    Warnings                                    
--------------------------------------------------------------------------------
§2.6 Attributes
* §2.6.2 global attribute title should exist and be a non-empty string


----------------------------------------------------------------

Using cached standard name table v76 from /home/oyvindl/.local/share/compliance-checker/cf-standard-name-table-test-76.xml


## Export to netCDF

In [207]:
D

<xarray.Dataset>
Dimensions:        (NISKIN_NUMBER: 11, TIME: 3)
Coordinates:
  * NISKIN_NUMBER  (NISKIN_NUMBER) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * TIME           (TIME) float64 1.863e+04 1.863e+04 1.863e+04
Data variables:
    PSAL           (NISKIN_NUMBER, TIME) float64 34.67 34.66 ... 34.25 33.96
    PRES           (NISKIN_NUMBER, TIME) float64 1.518e+03 2.22e+03 ... 9.782
    TEMP           (NISKIN_NUMBER, TIME) float64 0.2107 0.0278 ... -1.694 -1.654
    CNDC           (NISKIN_NUMBER, TIME) float64 2.964 2.976 ... 2.706 2.688
    STATION        (TIME) <U4 ' 012' ' 014' ' 015'
    LATITUDE       (TIME) float64 -69.42 -69.06 -69.25
    LONGITUDE      (TIME) float64 6.182 6.048 6.154
    CRUISE         <U12 '!! CRUISE !!'
Attributes: (12/45)
    history:                         2021-01-04 to 2021-01-05: Data collectio...
    cruise_name:                     DML 2020
    institution:                     Norwegian Polar Institute (NPI)
    source:                          CTD profiles
    instrument:                      SBE 911+
    data_set_language:               eng
    ...                              ...
    instrument_vocabulary:           NASA/GCMD Instrument Keywords Version 17.0
    keywords_vocabulary:             NASA/GCMD Science Keywords 9.1.5
    iso_topic_category:              oceans
    Conventions:                     ACDD-1.3, CF-1.8
    naming_authority:                npolar.no
    ship:                            Malik Arctica

In [208]:
ctd.to_netcdf(D, './', file_name = 'test_export_btl.nc')

Updated history attribute. Current content:
---
2021-01-04 to 2021-01-05: Data collection.
2021-01-13: Processed to .btl using SBE software (details in "SBE_processing").
2023-11-20: Creation of this netcdf file.
---
Exported netCDF file as: ./test_export_btl.nc.nc
